<a href="https://colab.research.google.com/github/Boss-Darius/PMP-2023/blob/main/PMP_Lab_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade pymc3
!pip uninstall -y theano
!pip install Theano==1.0.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Theano: filename=Theano-1.0.5-py3-none-any.whl size=2668109 sha256=3a11bffdfbd20c33fc04443651a0e5a1393d2813f1b11acfa269a5d641b626cb
  Stored in directory: /root/.cache/pip/wheels/d9/e6/7d/2267d21a99e4ab8276f976f293b4ff23f50c9d809f4a216ebb
Successfully built Theano


In [3]:
#1)
%env MKL_THREADING_LAYER=GNU
!pip install pymc3

import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt

pm.set_tt_rng(42)
pm.tt.compile.mode.get_default_mode().message = 'ignore'

# Dummy model to avoid version check
with pm.Model():
    pm.Deterministic('dummy', 0)

# Generăm date observate
np.random.seed(42)
data = np.random.binomial(n=1, p=0.3, size=100)

# Definim modelul în PyMC cu distribuție Beta
with pm.Model() as model_beta:
    p = pm.Beta('p', alpha=2, beta=2)
    obs = pm.Bernoulli('obs', p, observed=data)

# Folosim Metropolis pentru a estima parametrii
with model_beta:
    trace_beta = pm.sample(10000, tune=1000)

# Trasăm distribuția posterioră
pm.plot_posterior(trace_beta, var_names=['p'])
plt.title('Posterior - Distribuție Beta')
plt.show()


env: MKL_THREADING_LAYER=GNU
  Using cached pymc3-3.11.5-py3-none-any.whl (872 kB)
  Using cached scipy-1.7.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (39.9 MB)
INFO: pip is looking at multiple versions of arviz to determine which version is compatible with other requirements. This could take a while.
  Using cached arviz-0.16.1-py3-none-any.whl (1.7 MB)
  Using cached arviz-0.16.0-py3-none-any.whl (1.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.3 MB/s eta 0:00:00
  Using cached arviz-0.15.0-py3-none-any.whl (1.6 MB)
  Using cached arviz-0.14.0-py3-none-any.whl (1.7 MB)
  Using cached arviz-0.13.0-py3-none-any.whl (1.7 MB)
  Using cached arviz-0.12.1-py3-none-any.whl (1.6 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.11.4
    Uninstalling scipy-1.11.4:
      Successfully uninstalled scipy-1.11.4
  Attempting uninstall: arviz
    Found existing installation: arviz 0.17.0
    Uninstalling arviz-0.17.0:
      Successfully 

AttributeError: module 'theano' has no attribute 'config'

In [ ]:
#2)
def estimate_pi(N):
    x = np.random.uniform(-1, 1, N)
    y = np.random.uniform(-1, 1, N)
    inside_circle = np.sqrt(x**2 + y**2) <= 1
    pi_estimate = 4 * np.sum(inside_circle) / N
    error = np.abs(pi_estimate - np.pi)
    return error

# Fixăm N și calculăm eroarea de mai multe ori
N_values = [100, 1000, 10000]
errors = []

for N in N_values:
    error_values = [estimate_pi(N) for _ in range(100)]
    mean_error = np.mean(error_values)
    std_error = np.std(error_values)
    errors.append((N, mean_error, std_error))

# Vizualizăm rezultatele cu plt.errorbar()
plt.errorbar([item[0] for item in errors], [item[1] for item in errors], yerr=[item[2] for item in errors], fmt='o')
plt.xscale('log')
plt.xlabel('N')
plt.ylabel('Eroare')
plt.title('Eroarea în funcție de N')
plt.show()


In [ ]:
#3)
# Definim funcția pentru distribuția a priori beta-binomial
def log_prior(theta):
    alpha = 2
    beta = 2
    return (alpha - 1) * np.log(theta) + (beta - 1) * np.log(1 - theta)

# Definim funcția pentru log-likelihood binomial
def log_likelihood(theta, data):
    return np.sum(data * np.log(theta) + (1 - data) * np.log(1 - theta))

# Definim funcția log-posterior
def log_posterior(theta, data):
    return log_prior(theta) + log_likelihood(theta, data)

# Folosim Metropolis pentru a estima parametrii
def metropolis(log_posterior, theta0, n_samples, data):
    samples = [theta0]
    theta_current = theta0

    for _ in range(n_samples):
        theta_proposed = np.random.normal(theta_current, 0.1)
        alpha = np.exp(log_posterior(theta_proposed, data) - log_posterior(theta_current, data))
        if np.random.uniform(0, 1) < alpha:
            theta_current = theta_proposed
        samples.append(theta_current)

    return np.array(samples)

# Generăm date observate
np.random.seed(42)
data = np.random.binomial(n=1, p=0.3, size=100)

# Rulăm Metropolis
theta_samples = metropolis(log_posterior, theta0=0.5, n_samples=10000, data=data)

# Trasăm distribuția posterioră
plt.hist(theta_samples, bins=30, density=True, alpha=0.5, label='Metropolis')
plt.xlabel('Theta')
plt.ylabel('Densitatea de probabilitate')
plt.legend()
plt.show()
